In [1]:
import pandas as pd
from scipy import stats
from random import sample 
from tqdm import tqdm
from pathlib import Path
import pickle


In [2]:
def load_data():
    """
    """
    if Path("ts_data.pickle").is_file():
        with open('ts_data.pickle', 'rb') as handle:
            ts_data = pickle.load(handle)
    return ts_data


def color_normal(x):
    """
    """
    
    val = x.norm
    color = ''
    if val <= 0.0:
        color = 'green'
    if val > 0.0 and val <= 0.25:
        color = '#b1db57'
    elif val > 0.25 and val <= 0.5:
        color = '#d6db57'
    elif val > 0.5 and val <= 0.75:
        color = 'yellow'
    elif val > 0.75 and val <= 0.95:
        color = '#db9657'
    elif val >= 0.95:
        color = 'red'
        
    return ['background-color: %s' % color]*len(columns)



def get_temporal_summary(ts_data, symbol, ndays):
    """
    """
    
    data, meta_data = ts_data[symbol]
    symbol, market = symbol.split("_")
    
    updated_data = meta_data['3. Last Refreshed']
    current_price = data['5. adjusted close'][0]
    min_price = min(data['5. adjusted close'][:ndays])
    max_price = max(data['5. adjusted close'][:ndays])
    std_price = data['5. adjusted close'][:7].std()
    pmin_price = 100*((current_price/min_price)-1)
    pmax_price = 100*((current_price/max_price)-1)
    norm = (current_price - min_price)/(max_price - min_price)
    zscore = stats.zscore(data['5. adjusted close'][:ndays])[0]
    return symbol, market, updated_data, current_price, min_price, max_price, std_price, pmin_price, pmax_price, zscore, norm


def current_price_normalization(npast_days, market="all"):
    # load the data 
    ts_data = load_data()
    
    # collect all the stock price analysis
    lst_results = []

    for symbol in ts_data.keys():
        lst_results.append(get_temporal_summary(ts_data, symbol, ndays=npast_days))

    df = pd.DataFrame.from_records(lst_results, columns=columns)
    
    # filter
    if market != "all":
        df = df[df["market"]==market]
    
    df = df.sort_values(by=["symbol"])
    return df.style.apply(color_normal, axis=1)

    
columns = ["symbol", "market", "updated_data", "current_price", "min", "max", "std", "pmin", "pmax", "zscore", "norm"]

### Looking for the stocks with lowest prices given a number of past days

In [6]:
current_price_normalization(npast_days=7, market="NDQ")

,symbol,market,updated_data,current_price,min,max,std,pmin,pmax,zscore,norm
25,AAPL,NDQ,2020-06-02,323.340000,316.730000,323.340000,2.379261,2.086951,0.000000,1.833408,1.000000
27,ADBE,NDQ,2020-06-02,390.440000,375.170000,390.440000,6.177987,4.070155,0.000000,1.235578,1.000000
31,AMRS,NDQ,2020-06-02,4.860000,3.170000,5.350000,0.829515,53.312303,-9.158879,1.203526,0.775229
36,BKNG,NDQ,2020-06-02,1655.000000,1631.300000,1746.910000,40.497271,1.452829,-5.261290,-0.404188,0.205000
37,BPY,NDQ,2020-06-02,10.420000,9.017700,10.811500,0.584835,15.550528,-3.621144,0.344550,0.781748
39,BPYU,NDQ,2020-06-02,10.570000,9.184500,11.114500,0.595942,15.085198,-4.899006,0.312314,0.717876
35,BYND,NDQ,2020-06-02,135.190000,120.820000,138.690000,6.906830,11.893726,-2.523614,0.681838,0.804141
43,DOYU,NDQ,2020-06-02,8.920000,6.910000,9.000000,0.780385,29.088278,-0.888889,0.650523,0.961722
6,EXPE,NDQ,2020-06-02,84.220000,77.950000,84.220000,2.330930,8.043618,0.000000,1.473572,1.000000
2,FB,NDQ,2020-06-02,232.720000,225.090000,234.910000,3.767443,3.389755,-0.932272,0.721254,0.776986
